In [1]:
!pip3 install pandas dash

In [2]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_e15ad934f7494b198e3c7d1f871fb1d3 = 'https://s3.eu.cloud-object-storage.appdomain.cloud'
else:
    endpoint_e15ad934f7494b198e3c7d1f871fb1d3 = 'https://s3.private.eu.cloud-object-storage.appdomain.cloud'

client_e15ad934f7494b198e3c7d1f871fb1d3 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Rg81YOdL3zDiG7O2l5ZR4WwGeoJgvkgMnDC5tFZHeqXB',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_e15ad934f7494b198e3c7d1f871fb1d3)

body = client_e15ad934f7494b198e3c7d1f871fb1d3.get_object(Bucket='applieddatasciencecapstonespacex-donotdelete-pr-zoxox4socyhajo',Key='spacex_launch_dash.xlsx')['Body']

df_data_0 = pd.read_excel(body.read())
df_data_0.head()
spacex_df = df_data_0

In [3]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
# spacex_df = pd.read_csv("C:/Prywatne/IBM Data Science/spacex_launch_dash.csv")
# spacex_df = pd.read_csv("spacex_launch_dash.csv") #spare lines for data retrieve, data have been downloaded using above Watson's API

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

/tmp/wsuser/ipykernel_362/1575152262.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/tmp/wsuser/ipykernel_362/1575152262.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [ ]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                options=[{'label': 'All Sites', 'value': 'ALL'},
                                         {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                         {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                         {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                         {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                         value='ALL',
                                         placeholder='Select a Launch Site here.',
                                         searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                               min=min_payload,
                                               max=max_payload,
                                               step=1,
                                               value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value')
)                                
def get_pie_graph(chart):
    #added IF statement
    
    if chart == 'ALL':
        pie_fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches by Site')
    
    else:
        filtered_df = spacex_df[['Launch Site', 'Mission Outcome', 'class']]
        filtered_df = filtered_df[filtered_df['Launch Site']==chart]
        pie_fig = px.pie(filtered_df, values='class', 
        names='Mission Outcome', 
        title='Total Success Launches by Site')
    
    return pie_fig    



# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
               [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')]
)

def get_scatter_graph(chart, payload):
    
    filtered_df2 = spacex_df[(spacex_df['Payload Mass (kg)'] > payload[0]) & (spacex_df['Payload Mass (kg)'] <= payload[1])]

    
    if chart == 'ALL':
        scatter_fig= px.scatter(filtered_df2, x='Payload Mass (kg)',y='class',color='Booster Version Category',
                        title="Correlation between Payload and Success for All sites")
    else:
        filtered_df2 = filtered_df2[filtered_df2['Launch Site']==chart]
        filtered_df2 = filtered_df2[(filtered_df2['Payload Mass (kg)'] > payload[0]) & (filtered_df2['Payload Mass (kg)'] <= payload[1])]
        scatter_fig= px.scatter(filtered_df2,x='Payload Mass (kg)', y='class', color='Booster Version Category',
                        title='Correlation between Payload and Success for selected Site')
    return scatter_fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
